In [1]:
import pandas as pd

Import all the files

In [2]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [3]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [4]:
v_to_cart = pd.merge(visits, cart, how='left')
print(v_to_cart.head())

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time  
0                 NaT  
1 2017-01-26 14:44:00  
2 2017-08-20 08:31:00  
3                 NaT  
4                 NaT  


Step 3: How long is `visits_cart`?

In [5]:
length = v_to_cart.visit_time.count()
print(length)

2000


Step 4: How many timestamps are null for `cart_time`?

In [6]:
nulls_cart = v_to_cart.cart_time.isnull().sum()
print(nulls_cart)

1652


Step 5: What percentage only visited?

In [9]:
v_no_c = 100 * nulls_cart / length
print("Percent visited, but did not use cart: {}".format(v_no_c))
v_to_checkout = pd.merge(v_to_cart, checkout, how='outer')
nulls_checkout = v_to_checkout.checkout_time.isnull().sum()
print(nulls_checkout)

Percent visited, but did not use cart: 82.6
1774


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [10]:
sum_carts = v_to_checkout.cart_time.notnull().sum()
sum_checkouts = v_to_checkout.checkout_time.notnull().sum()
cart_to_checkout_pct = 100 * sum_checkouts / sum_carts
print("Percent of carts that reached checkout: {percent}".format(percent=cart_to_checkout_pct))

Percent of carts that reached checkout: 64.94252873563218


Step 7: Merge it all together

In [11]:
all_steps = pd.merge(v_to_checkout, purchase, how='outer')
print(all_steps)

                                   user_id          visit_time  \
0     943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1     0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2     6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3     6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4     a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   
...                                    ...                 ...   
2103  33913ac2-03da-45ae-8fc3-fea39df827c6 2017-03-25 03:29:00   
2104  4f850132-b99d-4623-80e6-6e61d003577e 2017-01-08 09:57:00   
2105  f0830b9b-1f5c-4e74-b63d-3f847cc6ce70 2017-09-07 12:56:00   
2106  b01bffa7-63ba-4cd3-9d93-eb1477c23831 2017-07-20 04:37:00   
2107  0336ca81-8d68-443f-9248-ac0b8ad147d5 2017-11-15 10:11:00   

               cart_time       checkout_time       purchase_time  
0                    NaT                 NaT                 NaT  
1    2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2    2

Step 8: % of users who got to checkout but did not purchase

In [12]:
sum_purchases = all_steps.purchase_time.notnull().sum()
checkout_to_purchase_pct = 100 * sum_checkouts / sum_purchases
print("Percent of checkouts that purchased: {}".format(checkout_to_purchase_pct))

Percent of checkouts that purchased: 89.68253968253968


Step 9: check each part of the funnel, let's print all 3 of them again

In [13]:
visit_to_cart_pct = 100 - v_no_c
print("Percent of visitors that put item(s) in cart: {}".format(visit_to_cart_pct))
print("Percent of carts that reached checkout: {}".format(cart_to_checkout_pct))
print("Percent of checkouts that purchased: {}".format(checkout_to_purchase_pct))

Percent of visitors that put item(s) in cart: 17.400000000000006
Percent of carts that reached checkout: 64.94252873563218
Percent of checkouts that purchased: 89.68253968253968


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [22]:
purchase_funnel = all_steps[all_steps.purchase_time.notnull()].reset_index()
print(purchase_funnel)

     index                               user_id          visit_time  \
0        1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
1       14  486480e2-98c3-4d51-8f4b-b1c07228ce84 2017-01-27 16:34:00   
2       48  3ccdaf69-2d30-40de-b083-51372881aedd 2017-01-08 20:21:00   
3       49  3ccdaf69-2d30-40de-b083-51372881aedd 2017-01-08 20:21:00   
4       65  ab0125fc-9493-4f59-ad70-24ad264a3a0c 2017-11-18 03:21:00   
..     ...                                   ...                 ...   
247   2083  d2cb350b-2201-4290-b2e0-84a8bf0d6883 2017-08-08 16:05:00   
248   2093  f46c88d0-2441-40a8-97fe-6841ff6f050d 2017-09-06 08:42:00   
249   2097  f783c680-1d9a-437d-9f45-7827299b78fa 2017-06-25 08:07:00   
250   2098  f783c680-1d9a-437d-9f45-7827299b78fa 2017-06-25 08:07:00   
251   2099  f783c680-1d9a-437d-9f45-7827299b78fa 2017-06-25 08:07:00   

              cart_time       checkout_time       purchase_time  
0   2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
1  

Step 11: examine the results

In [27]:
purchase_times = purchase_funnel.purchase_time - purchase_funnel.visit_time
purchase_funnel['time_to_purchase'] = purchase_funnel.purchase_time - purchase_funnel.visit_time
print(purchase_funnel)

     index                               user_id          visit_time  \
0        1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
1       14  486480e2-98c3-4d51-8f4b-b1c07228ce84 2017-01-27 16:34:00   
2       48  3ccdaf69-2d30-40de-b083-51372881aedd 2017-01-08 20:21:00   
3       49  3ccdaf69-2d30-40de-b083-51372881aedd 2017-01-08 20:21:00   
4       65  ab0125fc-9493-4f59-ad70-24ad264a3a0c 2017-11-18 03:21:00   
..     ...                                   ...                 ...   
247   2083  d2cb350b-2201-4290-b2e0-84a8bf0d6883 2017-08-08 16:05:00   
248   2093  f46c88d0-2441-40a8-97fe-6841ff6f050d 2017-09-06 08:42:00   
249   2097  f783c680-1d9a-437d-9f45-7827299b78fa 2017-06-25 08:07:00   
250   2098  f783c680-1d9a-437d-9f45-7827299b78fa 2017-06-25 08:07:00   
251   2099  f783c680-1d9a-437d-9f45-7827299b78fa 2017-06-25 08:07:00   

              cart_time       checkout_time       purchase_time  \
0   2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00   
1

Step 12: average time to purchase

In [26]:
avg_time_to_purchase = purchase_times.mean()
print("Average time from visit to purchase: {}".format(avg_time_to_purchase))

Average time from visit to purchase: 0 days 00:43:12.380952380
